In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

--2020-09-22 18:14:37--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.197.128, 209.85.200.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   141MB/s    in 0.6s    

2020-09-22 18:14:38 (141 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(3, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip
  
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip \
    -O /tmp/rps-test-set.zip

--2020-09-22 18:14:54--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rps.zip’

/tmp/rps.zip        100%[===================>] 191.38M   158MB/s    in 1.2s    

2020-09-22 18:14:56 (158 MB/s) - ‘/tmp/rps.zip’ saved [200682221/200682221]

--2020-09-22 18:14:56--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.145.128, 209.85.146.128, 209.85.234.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.145.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29516758 (28M) [application/zip]
Saving to: ‘

In [ ]:
import zipfile

local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

local_zip = '/tmp/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

In [ ]:
rock_dir = os.path.join('/tmp/rps/rock')
paper_dir = os.path.join('/tmp/rps/paper')
scissors_dir = os.path.join('/tmp/rps/scissors')

print('total training rock images:', len(os.listdir(rock_dir)))
print('total training paper images:', len(os.listdir(paper_dir)))
print('total training scissors images:', len(os.listdir(scissors_dir)))

total training rock images: 840
total training paper images: 840
total training scissors images: 840


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99):
      print("\n99% accuracy reached, cancelling training")
      self.model.stop_training = True

In [ ]:
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/tmp/rps/"
training_datagen = ImageDataGenerator(rescale = 1./255,
	                                    rotation_range = 40,
                                      width_shift_range = 0.2,
                                      height_shift_range = 0.2,
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      fill_mode = 'nearest')

VALIDATION_DIR = "/tmp/rps-test-set/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(TRAINING_DIR,
	                                                     target_size=(150,150),
	                                                     class_mode='categorical',
                                                       batch_size=126)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
	                                                            target_size=(150,150),
	                                                            class_mode='categorical',
                                                              batch_size=126)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [ ]:
callbacks = myCallback()
model.fit(train_generator, 
          epochs=20, 
          steps_per_epoch=20, 
          validation_data = validation_generator, 
          verbose = 1, 
          validation_steps=3, 
          callbacks = [callbacks])
model.save("rps.h5")

Epoch 1/20
20/20 [==============================] - 121s 6s/step - loss: 7.8917 - accuracy: 0.7290 - val_loss: 0.3683 - val_accuracy: 0.8602
Epoch 2/20
20/20 [==============================] - 123s 6s/step - loss: 1.2307 - accuracy: 0.8778 - val_loss: 0.1813 - val_accuracy: 0.9328
Epoch 3/20
20/20 [==============================] - 119s 6s/step - loss: 0.6437 - accuracy: 0.9040 - val_loss: 0.2870 - val_accuracy: 0.9140
Epoch 4/20
20/20 [==============================] - 119s 6s/step - loss: 0.5466 - accuracy: 0.9222 - val_loss: 0.3504 - val_accuracy: 0.8871
Epoch 5/20
20/20 [==============================] - 120s 6s/step - loss: 0.0632 - accuracy: 0.9778 - val_loss: 0.4186 - val_accuracy: 0.8495
Epoch 6/20
20/20 [==============================] - 119s 6s/step - loss: 0.5935 - accuracy: 0.9222 - val_loss: 0.1127 - val_accuracy: 0.9570
Epoch 7/20
20/20 [==============================] - 123s 6s/step - loss: 0.1111 - accuracy: 0.9694 - val_loss: 0.2730 - val_accuracy: 0.9167
Epoch 8/20
20

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print("Probability of paper, rock, scissors:")
  print(classes)

Saving img00.jpg to img00.jpg
Probability of paper, rock, scissors:
[[0. 1. 0.]]
